In [24]:
import requests
from bs4 import BeautifulSoup 
import re
from PIL import Image
from io import BytesIO
from twocaptcha import TwoCaptcha
from IPython.display import HTML
import sys

session = requests.Session()
home_url = "https://www.msmedatabank.in/Info/Verify.aspx"
home_response = session.get(home_url)
# print(home_response.text)
# sys.exit()
soup = BeautifulSoup(home_response.text, 'html.parser')

event_state_input = soup.find('input', {'id': '__VIEWSTATE'})
event_state_value = event_state_input.get('value', '') if event_state_input else ''

event_validation_input = soup.find('input', {'id': '__EVENTVALIDATION'})
event_validation_value = event_validation_input.get('value', '') if event_validation_input else ''

event_viewgen_input = soup.find('input', {'id': '__VIEWSTATEGENERATOR'})
event_viewgen_value = event_viewgen_input.get('value', '') if event_viewgen_input else ''

print("Event state value: ", event_state_value)
print("Event validation value: ", event_validation_value)
print("Event viewstate generator value:", event_viewgen_value)

middle_headers = {
  'Host': 'www.msmedatabank.in',
  'Origin': 'https://www.msmedatabank.in',
  'Referer': 'https://www.msmedatabank.in/Info/Verify.aspx',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
}

middle_payload = {
    '__EVENTTARGET': '',
    '__EVENTARGUMENT': '',
    '__LASTFOCUS': '',
    '__VIEWSTATE': event_state_value,
    '__VIEWSTATEGENERATOR': event_viewgen_value,
    '__EVENTVALIDATION': event_validation_value,
    'ctl00$MainContent$ddRegType':'UDYAM',
    'ctl00$MainContent$txCaptcha':'',

}

middle_response = session.post(home_url, data = middle_payload, headers=middle_headers)
soup_middle = BeautifulSoup(middle_response.text, 'html.parser')

event_state_input_middle = soup_middle.find('input', {'id': '__VIEWSTATE'})
event_state_value_middle = event_state_input_middle.get('value', '') if event_state_input else ''

event_validation_input_middle = soup_middle.find('input', {'id': '__EVENTVALIDATION'})
event_validation_value_middle = event_validation_input_middle.get('value', '') if event_validation_input else ''

event_viewgen_input_middle = soup_middle.find('input', {'id': '__VIEWSTATEGENERATOR'})
event_viewgen_value_middle = event_viewgen_input_middle.get('value', '') if event_viewgen_input else ''

print("Event state value: ", event_state_value_middle)
print("Event validation value: ", event_validation_value_middle)
print("Event viewstate generator value:", event_viewgen_value_middle)

regex_pattern_middle = r'CaptchaImage\.axd\?guid=([a-fA-F0-9\-]{36})'

match_middle = re.search(regex_pattern_middle, middle_response.text)
if match_middle:
    guid = match_middle.group(1)  # Use group(1) to get only the GUID part
    print(f"GUID: {guid}")
else:
    print("No GUID found.")
    

captcha_response = session.get(f"https://www.msmedatabank.in/Info/CaptchaImage.axd?guid={guid}")

captcha_image = Image.open(BytesIO(captcha_response.content))
captcha_image.save('captcha.png')
# captcha_image.show()

solver = TwoCaptcha('your_2captcha_key')
result = solver.normal('captcha.png')
captcha_value = result['code']

print("Captcha value: ", captcha_value)

udyam_number = 'UDYAM-MH-26-0229699'

# result_payload = f'_EVENTTARGET=&__EVENTARGUMENT=&__LASTFOCUS=&__VIEWSTATE={event_state_value}&__VIEWSTATEGENERATOR={event_viewgen_value}&__EVENTVALIDATION={event_validation_value}&ctl00%24MainContent%24ddRegType=UDYAM&ctl00%24MainContent%24txUAM={udyam_number}&ctl00%24MainContent%24txCaptcha={captcha_value}'

result_payload = {
    '__EVENTTARGET': '',
    '__EVENTARGUMENT': '',
    '__LASTFOCUS': '',
    '__VIEWSTATE': event_state_value_middle,
    '__VIEWSTATEGENERATOR': event_viewgen_value_middle,
    '__EVENTVALIDATION': event_validation_value_middle,
    'ctl00$MainContent$ddRegType':'UDYAM',
    'ctl00$MainContent$txUAM':udyam_number,
    'ctl00$MainContent$txCaptcha':captcha_value.upper(),
    'ctl00$MainContent$btnVerify': 'Verify Details'

}
print(result_payload)

result_headers = {
  'Host': 'www.msmedatabank.in',
  'Origin': 'https://www.msmedatabank.in',
  'Referer': 'https://www.msmedatabank.in/Info/Verify.aspx',
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
}

result_response = session.post(home_url, headers = result_headers, data = result_payload)
# print(display(HTML(result_response.text)))

with open('result.html', 'w', encoding='utf-8') as f:
    f.write(result_response.text)



Event state value:  /wEPDwUKMTk1NzA5OTg3Ng9kFgJmD2QWAgIDD2QWAgIBD2QWBgIBDxYCHgdWaXNpYmxlaBYCAgEPDxYCHgRUZXh0BQEuZGQCAw8QZGQWAWZkAhcPFgIfAGgWAgIZDzwrABECARAWABYAFgAMFCsAAGQYAgUhY3RsMDAkTWFpbkNvbnRlbnQkZ3JkVmlld0FjdGl2aXR5D2dkBRpjdGwwMCRNYWluQ29udGVudCRjYXB0Y2hhMQ8FJDY2YWQ1NmU4LWY5NDItNGFhMS05M2U4LTYzMDUxMDYyYzA5OWSTtQGS7btW7v57za/9gak+NJbk6xhhWYvO06dyRtOydg==
Event validation value:  /wEdAAgLat4TWHqOAPovxP2AbTmhaUcL2M/Rny0NqTVxJYZhEWnEH/jl33Z4nK8vIvW9ztIITDj9Y6/IJLVUVYUEwewDHXwsagaK6mbgPi1WQsDnF/t/9wWLIXlXlAoGT3UnAX8c0Su2YdopBVGMsb129XfTgzXoR1hTmy/Xi68C8UALY3IZZcfDKq6WchVrkSAyvdryr2UWrFG/x26XCCyOsYMb
Event viewstate generator value: DF529F84
Event state value:  /wEPDwUKMTk1NzA5OTg3Ng9kFgJmD2QWAgIDD2QWAgIBD2QWCgIBDxYCHgdWaXNpYmxlaBYCAgEPDxYCHgRUZXh0BQEuZGQCAw8QZGQWAQICZAIHDw8WAh8BBRpVZHlhbSBSZWdpc3RyYXRpb24gTnVtYmVyOmRkAgkPDxYEHglNYXhMZW5ndGgCEx4HRW5hYmxlZGdkZAIXDxYCHwBoFgICGQ88KwARAwAPFgQeC18hRGF0YUJvdW5kZx4LXyFJdGVtQ291bnRmZAEQFgAWABYADBQrAABkGAIFIWN0bDAwJE1haW5Db250ZW50JGdyZFZpZXdBY3R

Activity,2 Digit Activity,4 Digit Activity,5 Digit Activity
Manufacturing,"26-Manufacture of computer, electronic and optical products","2651-Manufacture of measuring, testing, navigating and control equipment",26517-Manufacture of industrial process control equipment
Services,"62-Computer programming, consultancy and related activities",6201-Computer programming activities,"62011-Writing , modifying, testing of computer program to meet the needs of a particular client excluding web-page designing"


None
